In [60]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import pandas as pd
import time



In [2]:
chromedriver_location = "chrome_driver/chromedriver"

In [3]:
# xPaths to each required search element in the field. 
district_drop_down = '//*[@id="courtId"]'
date_field = '//*[@id="inputVO.startDate"]'
seven_day_search_radio = '//*[@id="sevenDayRadio"]'
search_by_attorney_radio = '//*[@id="searchAttorneyRadio"]'
mobar_input = '//*[@id="inputVO.mobarNumber"]'
submit_button = '//*[@id="findButton"]'

In [4]:
# create an instance of the webdriver and load the inital calendar search page
driver = webdriver.Chrome(chromedriver_location)
driver.get('https://www.courts.mo.gov/casenet/cases/calendarSearch.do')

# store the district dropdown in an object using Select
courtDistrictDrp = Select(driver.find_element_by_xpath(district_drop_down))
courtDistrictDrp.select_by_value("CT11") # 11th District Saint Charles

# date
driver.find_element_by_xpath(date_field).send_keys("8/10/2021")
# 7 day search results
driver.find_element_by_xpath(seven_day_search_radio).click()
# search by attorney
driver.find_element_by_xpath(search_by_attorney_radio).click()
# MOBAR number field 
driver.find_element_by_xpath(mobar_input).send_keys("71855")

# without the pause, the inputs are too fast for casenet to handle
time.sleep(1) 

# submit 
driver.find_element_by_xpath(submit_button).click()



In [5]:
headers = driver.find_elements_by_xpath('//td[@class="header"]')

In [9]:
for h in headers:
    print(h.text)

 Tuesday, August 10, 2021
Case Number
Style of Case
Time
Day
Setting
Event
 Wednesday, August 11, 2021
Case Number
Style of Case
Time
Day
Setting
Event


In [27]:
i=1
data = []
while driver.find_elements_by_xpath(f'//td[@class="td{i}"]'):
    data.append(driver.find_elements_by_xpath(f'//td[@class="td{i}"]'))
    i += 1



In [31]:
for d in data:
    for i in d:
        print(i.text)

1611-CR00238-01  
ST V RACHEL N HARMON 
9:00 AM 
1 OF 1
 
Disposition Hearing 
Event Text:  
Location: DIVISION 1 COURTROOM St Charles Circuit Div
Judge/Commissioner: REBECA NAVARRO-MCKELVEY 
1911-CR03057-01  
ST V JOHN O'SULLIVAN 
9:00 AM 
1 OF 1
 
Motion Hearing 
Event Text:  
Location: DIVISION 1 COURTROOM St Charles Circuit Div
Judge/Commissioner: REBECA NAVARRO-MCKELVEY 
1911-CR01083-01  
ST V JOHN O'SULLIVAN 
9:00 AM 
1 OF 1
 
Motion Hearing 
Event Text: 
Location: DIVISION 1 COURTROOM St Charles Circuit Div
Judge/Commissioner: REBECA NAVARRO-MCKELVEY
2011-CR03048-01  
ST V JACOB SHAMUS THOMURE 
9:00 AM 
1 OF 1
 
Motion Hearing 
Event Text: MOTION TO WITHDRAW
Location: DIVISION 2 COURTROOM St Charles Circuit Div
Judge/Commissioner: DEBORAH JEAN ALESSI


In [80]:
table = driver.find_element_by_xpath('/html/body/table/tbody/tr[2]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td/table')

In [85]:
table_html = table.get_attribute('outerHTML')

In [97]:
df = pd.read_html(table_html)[0]

In [98]:
df

,0,1,2,3,4,5,6,7
0,NaN,"Tuesday, August 10, 2021","Tuesday, August 10, 2021","Tuesday, August 10, 2021","Tuesday, August 10, 2021","Tuesday, August 10, 2021","Tuesday, August 10, 2021","Tuesday, August 10, 2021"
1,NaN,Case Number,Style of Case,Style of Case,Time,Day,Setting,Event
2,NaN,1611-CR00238-01,ST V RACHEL N HARMON,ST V RACHEL N HARMON,9:00 AM,1 OF 1,NaN,Disposition Hearing
3,NaN,Event Text:,Event Text:,Event Text:,Event Text:,Event Text:,Location: DIVISION 1 COURTROOM St Charles Circ...,Location: DIVISION 1 COURTROOM St Charles Circ...
4,NaN,Judge/Commissioner: REBECA NAVARRO-MCKELVEY,Judge/Commissioner: REBECA NAVARRO-MCKELVEY,Judge/Commissioner: REBECA NAVARRO-MCKELVEY,Judge/Commissioner: REBECA NAVARRO-MCKELVEY,Judge/Commissioner: REBECA NAVARRO-MCKELVEY,Judge/Commissioner: REBECA NAVARRO-MCKELVEY,Judge/Commissioner: REBECA NAVARRO-MCKELVEY
5,NaN,1911-CR01083-01,ST V JOHN O'SULLIVAN,ST V JOHN O'SULLIVAN,9:00 AM,1 OF 1,NaN,Motion Hearing
6,NaN,Event Text:,Event Text:,Event Text:,Event Text:,Event Text:,Location: DIVISION 1 COURTROOM St Charles Circ...,Location: DIVISION 1 COURTROOM St Charles Circ...
7,NaN,Judge/Commissioner: REBECA NAVARRO-MCKELVEY,Judge/Commissioner: REBECA NAVARRO-MCKELVEY,Judge/Commissioner: REBECA NAVARRO-MCKELVEY,Judge/Commissioner: REBECA NAVARRO-MCKELVEY,Judge/Commissioner: REBECA NAVARRO-MCKELVEY,Judge/Commissioner: REBECA NAVARRO-MCKELVEY,Judge/Commissioner: REBECA NAVARRO-MCKELVEY
8,NaN,1911-CR03057-01,ST V JOHN O'SULLIVAN,ST V JOHN O'SULLIVAN,9:00 AM,1 OF 1,NaN,Motion Hearing
9,NaN,Event Text:,Event Text:,Event Text:,Event Text:,Event Text:,Location: DIVISION 1 COURTROOM St Charles Circ...,Location: DIVISION 1 COURTROOM St Charles Circ...
